# AutoGen for AI Agents

This notebook demonstrates how to use AutoGen to create and deploy AI agents with Azure AI services. AutoGen is an open-source framework that enables the creation of conversational AI applications using large language models (LLMs).

In this notebook, we'll:
1. Set up the AutoGen environment and dependencies
2. Configure Azure AI services integration
3. Create an AI agent with specific capabilities
4. Exchange messages with the agent
5. Process and display conversation results

AutoGen provides tools for building, customizing and deploying LLM applications with multiple agents that can converse with each other to solve tasks.

In [ ]:
!pip list

## Importing Required Libraries

We'll start by importing the necessary libraries to work with AutoGen and Azure AI services:

- `os`: For accessing environment variables
- `autogen_agentchat.agents`: Contains agent definitions including `AssistantAgent`
- `autogen_core.models`: For message handling and user interactions
- `autogen_ext.models.azure`: Azure AI integration for chat completion services
- `azure.core.credentials`: For secure authentication with Azure services
- `autogen_core`: Core AutoGen functionality including cancellation support
- `autogen_agentchat.messages`: Message types for agent communication
- `autogen_agentchat.ui`: UI components for displaying conversations

These libraries provide the foundation for creating agents, managing conversations, and integrating with Azure's AI services in a secure and scalable way.

In [ ]:
# Import necessary libraries for AutoGen agent-based chat applications# This includes Azure AI services integration for chat completions, agent definitions,# message handling, and UI components for building conversational AI applicationsimport osfrom autogen_agentchat.agents import AssistantAgentfrom autogen_core.models import UserMessagefrom autogen_ext.models.azure import AzureAIChatCompletionClientfrom azure.core.credentials import AzureKeyCredentialfrom autogen_core import CancellationTokenfrom autogen_agentchat.messages import TextMessagefrom autogen_agentchat.ui import Console

## Setting Up Azure AI Integration

In this section, we configure AutoGen to work with Azure AI services. Following Azure best practices for security:

1. We load environment variables from a `.env` file using `dotenv`
2. We authenticate with Azure AI services using a GitHub token (for demonstration purposes)
3. We create an `AzureAIChatCompletionClient` configured for the `gpt-4o-mini` model

The client is configured with the model's capabilities, including:
- JSON output support
- Function calling
- Vision capabilities

In a production environment, you should:
- Use Azure Managed Identities where possible instead of API keys
- Store sensitive credentials in Azure Key Vault
- Implement proper credential rotation and least privilege access
- Consider implementing retry logic for transient failures

To verify our configuration, we'll send a simple test question to the AI service.

In [ ]:
# This cell configures the Azure AI Chat Completion client# It loads environment variables, prints a GitHub token,# and creates a client instance to interact with the Azure OpenAI service# Finally it sends a test question to confirm the connection worksfrom dotenv import load_dotenv# Load environment variables from .env fileload_dotenv(override=True)print(os.environ["GITHUB_TOKEN"])client = AzureAIChatCompletionClient(    model="gpt-4o-mini",    endpoint="https://models.inference.ai.azure.com",    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),    model_info={        "json_output": True,        "function_calling": True,        "vision": True,        "family": "unknown",    },)result = await client.create([UserMessage(content="What is the capital of South Korea?", source="user")])print(result)

## Creating an AI Agent with AutoGen

Now we'll create an AI agent using AutoGen's `AssistantAgent` class. This approach offers several advantages:

1. **Abstraction**: AutoGen handles the complexity of agent interactions
2. **Integration**: Seamless connection with Azure AI services
3. **Extensibility**: Support for tools and multi-agent systems

Our agent is defined with:
- **Name**: A unique identifier ("assistant")
- **Model Client**: Our configured Azure AI Chat Completion client
- **Tools**: An empty array (no specialized tools for this example)
- **System Message**: Instructions that define the agent's role and behavior

The system message "You are a travel agent that plans great vacations" establishes the domain expertise of our agent. In production scenarios, you would typically provide more detailed instructions to control the agent's behavior, knowledge boundaries, and response patterns.

AutoGen's agent architecture allows you to add tools, skills, and advanced capabilities as your application grows in complexity.

In [ ]:
# Create an AssistantAgent for travel planning# This agent uses the Azure AI Chat Completion client we configured earlier# It will respond to user messages about travel planning with helpful suggestionsagent = AssistantAgent(    name="assistant",    model_client=client,    tools=[],    system_message="You are a travel agent that plans great vacations",)

## Interacting with the Agent

With our agent created, we can now interact with it by sending messages and processing responses. AutoGen provides an elegant asynchronous API for handling these interactions.

In this example, we'll:
1. Define an asynchronous function to handle the agent interaction
2. Create a user message asking for vacation planning advice
3. Send this message to our agent using the `on_messages` method
4. Process and display the agent's response

The `TextMessage` class represents a simple text-based message with content and source attributes. AutoGen supports more complex message types, including those with images or structured data.

The `CancellationToken` provides a way to cancel long-running operations if needed, which is important for responsiveness and resource management in production applications.

When the agent responds, we'll see two components:
- `inner_messages`: The detailed internal representation of the message
- `chat_message`: The formatted message ready for display

This separation allows for rich processing and presentation of agent responses.

In [ ]:
# This cell defines and runs an async function to demonstrate the travel agent assistant# It sends a message asking for vacation planning advice for Seoul and displays the responseasync def assistant_run() -> None:    response = await agent.on_messages(        [TextMessage(content="Could you help me plan a sunny vacation in Seoul, South Korea?", source="user")],        cancellation_token=CancellationToken(),    )    print(response.inner_messages)    print(response.chat_message)# Use asyncio.run(assistant_run()) when running in a script.await assistant_run()

## Summary and Next Steps

In this notebook, we've demonstrated how to use AutoGen with Azure AI services to create a conversational AI agent. We've covered:

1. **Setting up the environment** with necessary libraries and dependencies
2. **Configuring Azure AI integration** using secure authentication practices
3. **Creating an AI agent** with specific capabilities and instructions
4. **Exchanging messages** with the agent and processing responses

AutoGen provides a powerful framework for building conversational AI applications with Azure AI services. The integration with Azure follows security best practices and enables scalable, enterprise-grade AI solutions.

### Extending this Solution

You can enhance this example by:
- **Adding specialized tools** to the agent for performing specific tasks
- **Creating multi-agent systems** where agents collaborate to solve complex problems
- **Implementing persistent conversations** with storage and retrieval capabilities
- **Adding user authentication** and personalization features
- **Integrating with other Azure services** like Azure Functions, Logic Apps, or Cognitive Services

### Security Considerations

When deploying to production, consider:
- Using Azure Managed Identities instead of API keys where possible
- Storing sensitive credentials in Azure Key Vault
- Implementing proper RBAC (Role-Based Access Control) for your Azure AI resources
- Setting up monitoring and logging for agent activities
- Implementing content filtering and safety measures

For more information, visit the [AutoGen documentation](https://microsoft.github.io/autogen/) and [Azure AI services documentation](https://learn.microsoft.com/en-us/azure/ai-services/).